In [3]:
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

Cloning into 'models'...
remote: Enumerating objects: 49553, done.
remote: Total 49553 (delta 0), reused 0 (delta 0), pack-reused 49553
Receiving objects: 100% (49553/49553), 558.66 MiB | 33.04 MiB/s, done.
Resolving deltas: 100% (34184/34184), done.


In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from scipy.stats import itemfreq

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [5]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [6]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [7]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

     |████████████████████████████████| 49.5MB 85kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [13]:
#!wget https://storage.googleapis.com/videostestingobjectdetection/video_demo_baby_dog.mp4 

input='/test.mp4'

In [16]:
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (400,300))

filename = input
cap = cv2.VideoCapture(filename)

with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        counter = 0
        while (True):
            ret, image_np = cap.read()
            
            counter += 1
            if ret:
              h = image_np.shape[0]
              w = image_np.shape[1]

              if counter % 1 == 0:
                image_np_expanded = np.expand_dims(image_np, axis=0)
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')

                (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})

                vis_util.visualize_boxes_and_labels_on_image_array(image_np,
                                                                     np.squeeze(boxes),
                                                                     np.squeeze(classes).astype(np.int32),
                                                                     np.squeeze(scores),
                                                                     category_index,
                                                                     use_normalized_coordinates=True,
                                                                     line_thickness=5,
                                                                     min_score_thresh=0.3)

              out.write(image_np)
            else:
              break
out.release()
cap.release()
cv2.destroyAllWindows()

In [10]:
!pip install moviepy

In [11]:
import io
import base64
from IPython.display import HTML

# Before Object Detection

In [15]:
video = io.open(input, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="80%" alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

# After Object Detection

In [17]:
video = io.open('output.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="80%" video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))